# Introduction to Pandas

In this notebook we will go over some basics of the python module ```pandas``` using Python 3.5 (and above). 


## The toy dataset

We will use the dataset available [here](https://archive.ics.uci.edu/ml/datasets/Wine+Quality).
Read the information here to understand what the dataset is all about. 

The dataset contains information about the perceived quality of a particular brand of Portuguese wine.  

Each row will represent a particular red wine variant and each column will contain the following:
1. fixed acidity
2. volatile acidity
3. citric acid
4. residual sugar
5. chlorides
6. free sulfur dioxide
7. total sulfur dioxide
8. density
9. pH
10. sulphates
11. alcohol
12. quality (score between 0 and 10)

We will use the data to learn more about working with ```pandas```.


## Downloading the data:

Visit [this link](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv) and save the CSV file to the **same folder** where you have saved **this notebook** without altering the suggested name. 


This is a CSV (Comma Separated Values) file. Go ahead and open it with a **text editor**.

![title](csv_head.png)

You can see that these are not comma separated values and they are separated by a ```;```. Generally values in CSV files can be separated by various different characters such as space, commas, tabs, colons and semi-colons. 

In [ ]:
# Loading the pandas module with the name 'pd' 
import pandas as pd 

In [ ]:
redwine = pd.read_csv('winequality-red.csv')  # read the csv file

In [ ]:
redwine.head()  # display the first 5 rows of the CSV

It seems that something has broken... 


Normally when we use the ```.read_csv ``` method of pandas, it assumes that the delimiter between values is a ```,```. In cases where this is not (such as ours) we can explicitly specify the delimiter. 

In [ ]:
redwine = pd.read_csv('winequality-red.csv', sep=';')  # read the csv file separated with a semicolon ;

In [ ]:
redwine.head()

This is much more readable.

The object ```redwine``` is a ```pandas.DataFrame``` object which we will use very extensively during the course. 

Let's investigate what is inside this dataframe. 

In [ ]:
redwine.info()  # This prints out some basic stats about the data

From this summary we can see the names of our columns, the number of non-empty rows, and the type of each column. In our case they are all floating point numbers (```float64```) except the last one, the quality. This is an integer. 

Now let's visualize some of the data. 

We can plot the ```pH``` and the ```citric acid``` columns to see if there is something interesting there.
To plot we use the ```.plot()``` method of a ```pandas.DataFrame``` object. 

We have to specify what values we want plotted along the x and y axes. There are also many other options (see [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html)).


In [ ]:
# Plotting age on the x axis and the number of nodes on the y axis
redwine.plot(x='citric acid', y='pH')

This is not very useful.

Pandas has drawn a line between points where there should be none.
We a scatter plot. We can use ```pandas.plot.scatter``` instead of just ```pandas.plot```. 

In [ ]:
# The figure is also too small let's increase the size by adjusting the figsize (x_size, y_size) option
redwine.plot.scatter(x='citric acid', y='pH', figsize=(10, 8))

Intuitively, the ```pH``` level decreases as the concentration of ```citric acid``` increases. 

What about wine density? My theory is is that the more ```alcohol``` the lower the ```density``` since alcohol is generally lighter than fruit juice. 

In [ ]:
redwine.plot.scatter(x='alcohol', y='density', figsize=(10, 8))

It seems that there is some correlation here as the alcohol level goes up.

What if we wanted to see this effect but only for wines that are bad?

First we will have to find out what is bad wine. We will define it as wines that have a lower than mean quality score.  

Therefore we must first find the mean value of the ```quality``` column. In pandas we can select a column like we would the index of a list like so:

In [ ]:
redwine['quality']

These are all the column values. 

Pandas columns are ```pandas.Series``` objects and have most of the functionality of ```numpy arrays```. This means we can use mean, median and sum calculations. For now we will use the mean. 

In [ ]:
ave_quality = redwine['quality'].mean()

In [ ]:
print(ave_quality)

Now that we know the mean, how do we select only those rows that have a quality value less than ```ave_quality```?

We can slice a ```pandas.DataFrame``` object by indexing it with conditional statements. 
This is because a conditional statement on on a dataframe returns a series of boolean values.
See the example below. 

In [ ]:
# using a conditional statement on a pandas dataframe directly
redwine['quality'] < ave_quality

We can now pass this series of boolean values directly to a dataframe as we would a column. 
i.e. ```redwine[redwine['quality'] < ave_quality]``` returns only those rows in ```redwine``` where the value of the ```'quality'``` column is less than the value of ```ave_quality```. 

Therefore we can do the following:

In [ ]:
# new dataframe with only the data about the dead
badwine = redwine[redwine['quality'] < ave_quality]

In [ ]:
# Some summary statistics
badwine.info()

In [ ]:
# print the first 5 rows
badwine.head()

Now we will go back and plot this. 

In [ ]:
badwine.plot.scatter(x='alcohol', y='density', figsize=(10, 8))

It's kind of hard to visually compare this to what we had before. 

Ideally we could plot these in the same figure. 

This is possible. In pandas, every ```.plot``` statement returns an ```axis``` object which is a reference to figure that the plot is being drawn upon. 

Further, every ```.plot``` statement also takes as an optional input an axis object. Therefore we can do the following:

In [ ]:
my_canvas = redwine.plot.scatter(x='alcohol', y='density', figsize=(10, 8))
badwine.plot.scatter(x='alcohol', y='density', figsize=(10, 8), ax=my_canvas)

So what happened?

We have plotted both plots on top of each other, however, they have the same exact color. We will need to change the color of one of them. 

Let's set the color of the second plot to ```red```. 

In [ ]:
my_canvas = redwine.plot.scatter(x='alcohol', y='density', figsize=(10, 8))
badwine.plot.scatter(x='alcohol', y='density', figsize=(10, 8), ax=my_canvas, color='red')

Much more interesting. 

Not only do we find that ```density``` is related to ```alcohol``` but also that the red plot (bad wines) tend to all have a lower ```alcohol``` level. 

Perhaps ```alcohol``` is very important in wine quality.

We can look at just ```alcohol``` in histograms. 

First let's look at the ```alcohol``` distribution in all of our data. 

In [ ]:
redwine['alcohol'].plot.hist(figsize=(10, 8))

On the y axis we see the frequency of ```alcohol``` levels specified on the x axis. 

Now let's plot them on top of each other like before. 

In [ ]:
the_figure = redwine['alcohol'].plot.hist(figsize=(10, 8))
badwine['alcohol'].plot.hist(figsize=(10, 8), color='red', ax = the_figure)

This is not very informative. We want to compare ```alcohol``` levels _per-capita_. 

In other words, we want to normalise the frequency counts so that they represent **density** rather than frequency. 

The pandas ```.hist()``` method has an option ```density``` which we can set to true.

In [ ]:
the_figure = redwine['alcohol'].plot.hist(figsize=(10, 8), density=True)
badwine['alcohol'].plot.hist(figsize=(10, 8), color='red', density=True)

We can now see very clearly that the _bad wine_ clearly has a lower ```alcohol``` content as the red histogram (corresponding to bad wines) is tighter around lower ```alcohol``` values. 

Finally, we can make plots more transparent so that the underlying plots can be seen more easily by tuning the ```alpha``` parameter in most plotting functions in python. 

In [ ]:
the_figure = redwine['alcohol'].plot.hist(figsize=(10, 8), density=True)
badwine['alcohol'].plot.hist(figsize=(10, 8), color='red', density=True, alpha=0.5)